# Myers-Briggs Type Indicator (MBTI)

The Myers-Briggs Type Indicator (MBTI) is a popular personality assessment tool based on Carl Jung’s theory of psychological types. It categorizes individuals into one of 16 distinct personality types based on their preferences in four dichotomies:

## Dichotomies

1. **Introversion (I) vs. Extraversion (E)**
   - **Introverts (I)**: Energized by solitary activities, require time alone to recharge.
   - **Extraverts (E)**: Energized by interactions with others, seek out social engagement.


2. **Sensing (S) vs. Intuition (N)**
   - **Sensors (S)**: Focus on concrete, tangible details and present realities.
   - **Intuitives (N)**: Interested in abstract concepts and future possibilities.
   

3. **Thinking (T) vs. Feeling (F)**
   - **Thinkers (T)**: Prioritize logic and objective criteria in decision-making.
   - **Feelers (F)**: Base decisions on personal values and how they affect others.
   

4. **Judging (J) vs. Perceiving (P)**
   - **Judgers (J)**: Prefer structure, order, and decisiveness.
   - **Perceivers (P)**: More adaptable, spontaneous, and open to new information.

## Personality Types

Combining these preferences results in 16 distinct personality types:

1. **ISTJ (Introverted, Sensing, Thinking, Judging)**: The Inspector - Responsible, organized, and practical.

2. **ISFJ (Introverted, Sensing, Feeling, Judging)**: The Protector - Compassionate, detail-oriented, and loyal.

3. **INFJ (Introverted, Intuition, Feeling, Judging)**: The Advocate - Idealistic, insightful, and principled.

4. **INTJ (Introverted, Intuition, Thinking, Judging)**: The Architect - Strategic, analytical, and independent.

5. **ISTP (Introverted, Sensing, Thinking, Perceiving)**: The Virtuoso - Adventurous, logical, and hands-on.

6. **ISFP (Introverted, Sensing, Feeling, Perceiving)**: The Composer - Artistic, gentle, and spontaneous.

7. **INFP (Introverted, Intuition, Feeling, Perceiving)**: The Mediator - Creative, empathetic, and idealistic.

8. **INTP (Introverted, Intuition, Thinking, Perceiving)**: The Logician - Curious, analytical, and independent.

9. **ESTP (Extraverted, Sensing, Thinking, Perceiving)**: The Entrepreneur - Energetic, pragmatic, and action-oriented.

10. **ESFP (Extraverted, Sensing, Feeling, Perceiving)**: The Entertainer - Sociable, lively, and spontaneous.

11. **ENFP (Extraverted, Intuition, Feeling, Perceiving)**: The Campaigner - Enthusiastic, imaginative, and sociable.

12. **ENTP (Extraverted, Intuition, Thinking, Perceiving)**: The Debater - Innovative, energetic, and intellectually curious.

13. **ESTJ (Extraverted, Sensing, Thinking, Judging)**: The Executive - Organized, assertive, and pragmatic.

14. **ESFJ (Extraverted, Sensing, Feeling, Judging)**: The Consul - Caring, social, and organized.

15. **ENFJ (Extraverted, Intuition, Feeling, Judging)**: The Protagonist - Charismatic, empathetic, and organized.

16. **ENTJ (Extraverted, Intuition, Thinking, Judging)**: The Commander - Confident, strategic, and decisive.

In [6]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from catboost import CatBoostClassifier
import category_encoders as ce

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

In [2]:
train = pd.read_csv("/kaggle/input/predict-people-personality-types/data.csv")
train = train.loc[train['Interest'] != 'Unknown']
train

,Age,Gender,Education,Introversion Score,Sensing Score,Thinking Score,Judging Score,Interest,Personality
1,27.0,Female,0,5.85392,6.160195,0.80552,4.221421,Sports,ESFP
3,28.0,Male,0,2.01892,4.823624,7.30625,5.986550,Others,INTP
4,36.0,Female,1,9.91703,4.755080,5.31469,4.677213,Technology,ENFP
5,25.0,Male,0,7.66292,5.758030,1.97014,4.563907,Others,ENFP
7,27.0,Female,0,5.24958,5.963290,1.16176,5.873350,Sports,ENFJ
...,...,...,...,...,...,...,...,...,...
128054,24.0,Female,0,2.62070,5.871295,3.15414,6.237609,Technology,INFJ
128055,38.0,Male,1,5.18969,5.511788,9.69071,4.456348,Technology,ENTP
128056,27.0,Male,0,5.11456,7.764336,5.72341,2.686904,Arts,ESTP
128058,26.0,Female,1,8.16219,3.433857,2.45531,5.541144,Arts,ENFP


In [3]:
encoder = ce.OrdinalEncoder(
    mapping=[
        {'col': 'Gender', 'mapping': {'Female': 1, 'Male': 0}},
        {'col': 'Interest', 'mapping': {'Arts': 0, 'Sports': 1, 'Technology': 2, 'Others': 3}},
    ]
)

# Apply encoding to the dataset
train_encoded = encoder.fit_transform(train)

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    train_encoded.drop(labels=['Personality'], axis=1),
    train_encoded['Personality'],
    test_size=0.3,
    random_state=0
)

# Defining and training the CatBoost model
model = CatBoostClassifier(
    loss_function='MultiClass',
    verbose=False,
    n_estimators=300,
    max_bin=300,
    min_data_in_leaf=100,
    l2_leaf_reg=10
)
model.fit(X_train, y_train)

# Making predictions
y_pred = model.predict(X_test)

# Printing the classification report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

        ENFJ       0.92      0.90      0.91       812
        ENFP       0.92      0.93      0.93      6178
        ENTJ       0.89      0.92      0.91       586
        ENTP       0.92      0.93      0.93      4630
        ESFJ       0.78      0.77      0.77       112
        ESFP       0.87      0.82      0.84       882
        ESTJ       0.82      0.71      0.76        86
        ESTP       0.85      0.82      0.83       672
        INFJ       0.89      0.89      0.89       593
        INFP       0.90      0.91      0.90      4415
        INTJ       0.90      0.90      0.90       413
        INTP       0.90      0.90      0.90      3188
        ISFJ       0.74      0.72      0.73        71
        ISFP       0.83      0.81      0.82       641
        ISTJ       0.74      0.56      0.64        52
        ISTP       0.82      0.78      0.80       437

    accuracy                           0.90     23768
   macro avg       0.86   

In [4]:
mbti_types = {
    "ISTJ": {
        "type": "Introverted, Sensing, Thinking, Judging",
        "nickname": "The Inspector",
        "traits": ["Responsible", "Organized", "Practical"]
    },
    "ISFJ": {
        "type": "Introverted, Sensing, Feeling, Judging",
        "nickname": "The Protector",
        "traits": ["Compassionate", "Detail-oriented", "Loyal"]
    },
    "INFJ": {
        "type": "Introverted, Intuition, Feeling, Judging",
        "nickname": "The Advocate",
        "traits": ["Idealistic", "Insightful", "Principled"]
    },
    "INTJ": {
        "type": "Introverted, Intuition, Thinking, Judging",
        "nickname": "The Architect",
        "traits": ["Strategic", "Analytical", "Independent"]
    },
    "ISTP": {
        "type": "Introverted, Sensing, Thinking, Perceiving",
        "nickname": "The Virtuoso",
        "traits": ["Adventurous", "Logical", "Hands-on"]
    },
    "ISFP": {
        "type": "Introverted, Sensing, Feeling, Perceiving",
        "nickname": "The Composer",
        "traits": ["Artistic", "Gentle", "Spontaneous"]
    },
    "INFP": {
        "type": "Introverted, Intuition, Feeling, Perceiving",
        "nickname": "The Mediator",
        "traits": ["Creative", "Empathetic", "Idealistic"]
    },
    "INTP": {
        "type": "Introverted, Intuition, Thinking, Perceiving",
        "nickname": "The Logician",
        "traits": ["Curious", "Analytical", "Independent"]
    },
    "ESTP": {
        "type": "Extraverted, Sensing, Thinking, Perceiving",
        "nickname": "The Entrepreneur",
        "traits": ["Energetic", "Pragmatic", "Action-oriented"]
    },
    "ESFP": {
        "type": "Extraverted, Sensing, Feeling, Perceiving",
        "nickname": "The Entertainer",
        "traits": ["Sociable", "Lively", "Spontaneous"]
    },
    "ENFP": {
        "type": "Extraverted, Intuition, Feeling, Perceiving",
        "nickname": "The Campaigner",
        "traits": ["Enthusiastic", "Imaginative", "Sociable"]
    },
    "ENTP": {
        "type": "Extraverted, Intuition, Thinking, Perceiving",
        "nickname": "The Debater",
        "traits": ["Innovative", "Energetic", "Intellectually curious"]
    },
    "ESTJ": {
        "type": "Extraverted, Sensing, Thinking, Judging",
        "nickname": "The Executive",
        "traits": ["Organized", "Assertive", "Pragmatic"]
    },
    "ESFJ": {
        "type": "Extraverted, Sensing, Feeling, Judging",
        "nickname": "The Consul",
        "traits": ["Caring", "Social", "Organized"]
    },
    "ENFJ": {
        "type": "Extraverted, Intuition, Feeling, Judging",
        "nickname": "The Protagonist",
        "traits": ["Charismatic", "Empathetic", "Organized"]
    },
    "ENTJ": {
        "type": "Extraverted, Intuition, Thinking, Judging",
        "nickname": "The Commander",
        "traits": ["Confident", "Strategic", "Decisive"]
    }
}
arr = [27, 0, 1, 4, 6, 8, 8, 2]
print_dict = mbti_types[model.predict(arr)[0]]
for key, value in print_dict.items():
    print(key, ':', value)

type : Introverted, Intuition, Thinking, Perceiving
nickname : The Logician
traits : ['Curious', 'Analytical', 'Independent']


In [ ]:
import pickle

file = open('model.pkl', 'wb')
pickle.dump(model, file)